# Задание 1

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [1]:
corpus_wsd = []
corpus = open('data/corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [3]:
import nltk
from nltk.corpus import wordnet as wn

In [5]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [37]:
#Создаем список слов для сравнения
disamb = list()
for sentence in corpus_wsd:
    for word in sentence:
        if word[0] != '':
            disamb.append(word[1])

#disamb

In [40]:
#Создаем список ключей к определениям в wn
keys = list()
for sentence in corpus_wsd:
    for word in sentence:
        if word[0] != '':
            keys.append(word[0])

In [ ]:
#Создаем список контекстов для сравнения
contexts = list()
for sentence in corpus_wsd:
    for word in sentence:
        if word[0] != '':
            context = list()
            x = word[2]
            for word in sentence:
                if word[2] != x:
                    context.append(word[2])
                else:
                    context.append('_')

            context = ' '.join(context)
            contexts.append(context)

#contexts

In [ ]:
x = 'employee'
for synset in wn.synsets(x):
    print(word + ' - ' + synset.definition())

In [29]:
print(disamb[:20])

['long', 'be', 'review', 'objective', 'benefit', 'service', 'program', 'permit', 'become', 'giveaway', 'program', 'rather', 'have', 'goal', 'improved', 'employee', 'morale', 'consequently', 'increased', 'productivity']


In [ ]:
#Достать список определений 
#для каждого слова из списка слов для сравнения

all_definitions = list()
for word in disamb:
    definitions = list()
    for synset in wn.synsets(word):
        definitions.append(synset.definition())
    all_definitions.append(definitions)

#all_definitions

In [ ]:
#Получаем список наиболее подходящих определений по Jaccard

jaccard_defs = list()

for definitions in all_definitions:
    count = 0
    max = float(0)
    defs = list()
    for definition in definitions:
        intersection = (set(definition.split()) & set(contexts[count].split()))
        union = (set(definition.split()) | set(contexts[count].split()))
        jaccard = len(intersection) / len(union)
        if jaccard > max:
            max = jaccard
            defs.append(definition)
    if defs != []:
        jaccard_defs.append(defs[-1])
    else:
        jaccard_defs.append(defs)
    count += 1

jaccard_defs

In [34]:
jaccard_defs[0]

'planning prudently for the future'

In [56]:
#Сравниваем полученные определения с данными в корпусе
counter = 0
volume = len(disamb)
count = 0
for definition in jaccard_defs:
    lemma = wn.lemma_from_key(keys[count]).synset().definition()
    if definition == lemma:
        counter += 1

    count += 1


In [55]:
len(jaccard_defs)


239913

In [52]:
len(contexts)

239913

In [51]:
len(keys)

239913

In [50]:
volume

239913

In [59]:
#Считаем точность
accuracy = format(counter/volume, '.0%')
accuracy

'23%'

Теперь с косинусной близостью

In [8]:
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer

In [9]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embed = model.encode

In [72]:
definition00 = 'a system of projects or services intended to meet a public need'
definition02 = '(computer science) a sequence of instructions that a computer can interpret and execute'
context00 = 'i wrote a computer _ that predicts the sense of a word'
cossim00 = cosine_similarity(embed(definition00).reshape(1, -1), embed(context00).reshape(1, -1))
cossim02 = cosine_similarity(embed(definition02).reshape(1, -1), embed(context00).reshape(1, -1))
cossim00


array([[0.10057071]], dtype=float32)

In [73]:
cossim02

array([[0.3182025]], dtype=float32)

In [74]:
if cossim00 > cossim02:
    print('Larger')
elif cossim02 > cossim00:
    print ('Smaller')
else:
    print('EQ')


Smaller


In [ ]:
#Берем определение по наибольшей косинусной близости
cos_defs = list()

for definitions in all_definitions:
    count2 = 0
    max2 = float(0)
    defs2 = list()
    for definition in definitions:
        cossim = cosine_similarity(embed(definition).reshape(1, -1), embed(contexts[count2]).reshape(1, -1))
        
        if cossim > max2:
            max2 = cossim
            defs2.append(definition)
    if defs2 != []:
        cos_defs.append(defs2[-1])
    else:
        cos_defs.append(defs2)
    count2 += 1

cos_defs

In [ ]:
counter2 = 0
volume2 = len(disamb)
count3 = 0
for definition in cos_defs:
    lemma = wn.lemma_from_key(keys[count3]).synset().definition()
    if definition == lemma:
        counter2 += 1

    count3 += 1

In [ ]:
#Считаем точность тут
accuracy2 = format(counter2/volume, '.0%')
accuracy2

# Задание 2
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/train.csv', sep='\t')

In [ ]:
df.head(100)

In [4]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [ ]:
for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")
    break

In [6]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation
import numpy as np
from sklearn.metrics import adjusted_rand_score

from IPython.display import Image
from IPython.core.display import HTML

Смотрим KMeans

In [10]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(3)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.04802335780315568


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [11]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(5)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.07730266389650145


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [12]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(10)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.05481698793321215


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [13]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(7)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.06159488872721862


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [14]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    # cluster = AffinityPropagation(damping=0.9)
    cluster = KMeans(6)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.04013704407686531


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Смотрим AffinityPropagation


In [15]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    cluster = AffinityPropagation(damping=0.9)
    #cluster = KMeans(8)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.05297560306165972


In [16]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    cluster = AffinityPropagation(damping=0.5, max_iter=200, convergence_iter=15)
    #cluster = KMeans(8)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.042740969848549505


In [17]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    cluster = AffinityPropagation(damping=0.7, max_iter=100, convergence_iter=15)
    #cluster = KMeans(8)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.04150923434928373


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [18]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    cluster = AffinityPropagation(damping=0.7, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.04150923434928373


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_affinity_propagation.py:142: ConvergenceWarning: Affinity propagation did not converge, this model may return degenerate cluster centers and labels.
  warnings.warn(


In [20]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    # cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.040604690088052654


Смотрим DBSCAN


In [21]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    cluster = DBSCAN(min_samples=1, eps=0.1)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.001053019960000099


In [22]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    cluster = DBSCAN(eps=0.3, min_samples=5, metric='euclidean', algorithm='auto', leaf_size=30)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

-0.02349153051130162


In [25]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    cluster = DBSCAN(eps=0.3, min_samples=5, metric='manhattan')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.0


In [26]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    cluster = DBSCAN(eps=0.3, min_samples=5, metric='cosine')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

-0.001784882962278153


In [27]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    cluster = DBSCAN(eps=0.2, min_samples=1, metric='euclidean')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.00443342207693399


Смотрим на Agglomerative clustering

In [24]:
from sklearn.cluster import AgglomerativeClustering

In [28]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.3, min_samples=5, metric='euclidean')
    cluster = AgglomerativeClustering(n_clusters=3, linkage='ward', affinity='euclidean')

    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


0.034738071269679226


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [29]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.3, min_samples=5, metric='euclidean')
    cluster = AgglomerativeClustering(n_clusters=3, linkage='complete', affinity='euclidean')

    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


0.11591119562417704


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [30]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.3, min_samples=5, metric='euclidean')
    cluster = AgglomerativeClustering(n_clusters=3, linkage='average', affinity='euclidean')

    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


-0.009158236277409101


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [31]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.3, min_samples=5, metric='euclidean')
    cluster = AgglomerativeClustering(n_clusters=3, linkage='single', affinity='euclidean')

    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


0.0519783194524109


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [32]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.3, min_samples=5, metric='euclidean')
    cluster = AgglomerativeClustering(n_clusters=10, linkage='complete', affinity='euclidean')

    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


0.0716318676086122


c:\Users\Dmitry\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


Смотрим на пятый алгоритм на выбор - Spectral clustering

Как работает? Создав матрицу, алгоритм упрощает её, уменьшая количество измерений, но сохраняя более релевантные параметры. Потом, алгоритм делит элементы на заданное число кластеров. Инструкция уверяет, что данный алгоритм особенно хорошо работает с делением на два кластера. Он пытается разделить данные на две группы таким образом, чтобы минимизировать связи между группами и максимизировать связи внутри каждой группы.

In [33]:
from sklearn.cluster import SpectralClustering

In [35]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.2, min_samples=1, metric='euclidean')
    cluster = SpectralClustering(n_clusters=2)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.029294289618017777


In [36]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.2, min_samples=1, metric='euclidean')
    cluster = SpectralClustering(n_clusters=2, affinity='nearest_neighbors')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.03880940196533622


In [38]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.2, min_samples=1, metric='euclidean')
    cluster = SpectralClustering(n_clusters=3)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.057696021574798674


In [39]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.2, min_samples=1, metric='euclidean')
    cluster = SpectralClustering(n_clusters=3, affinity='nearest_neighbors')
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.08339604125532275


In [40]:
ARI = []

for key, _ in grouped_df:
    # вытаскиваем контексты
    texts = grouped_df.get_group(key)['context'].values

    # создаем пустую матрицу для векторов 
    X = np.zeros((len(texts), 768))

    # переводим тексты в векторы и кладем в матрицу
    for i, text in enumerate(texts):
        X[i] = embed(text)

    # выбираем один из алгоритмов
    #cluster = AffinityPropagation(damping=0.9, max_iter=100, convergence_iter=10)
    #cluster = KMeans(8)
    #cluster = DBSCAN(eps=0.2, min_samples=1, metric='euclidean')
    cluster = SpectralClustering(n_clusters=5)
    
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1 

    # расчитываем метрику для отдельного слова
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
print(np.mean(ARI)) # усредненная метрика

0.06722011006535526
